# 04: Guardrails

Implement safety guardrails.

**Learning Objectives:**
- Input validation
- PII detection
- Content filtering
- Output safety checks


In [ ]:
import sys
sys.path.insert(0, '..')

from src.guardrails import GuardrailsManager

print('✓ Imports successful')

## Input Validation

TODO: Add more validation rules

In [ ]:
guardrails = GuardrailsManager(strict_mode=False)

# Test validation
test_queries = [
    'What are GDPR rights?',
    'My email is test@example.com',
    '',  # Empty
]

for query in test_queries:
    is_valid, sanitized, error = guardrails.apply_guardrails(query)
    print(f'Query: {query[:50]}')
    print(f'Valid: {is_valid}, Error: {error}\n')

## PII Detection and Redaction

In [ ]:
text = 'Contact me at john@example.com or 555-123-4567'
detected = guardrails.detect_pii(text)
redacted, counts = guardrails.redact_pii(text)

print(f'Original: {text}')
print(f'Detected PII: {detected}')
print(f'Redacted: {redacted}')